Week 8 Notebook: Extending the Model
===============================================================

This week, we will look at graph neural networks using the PyTorch Geometric library: <https://pytorch-geometric.readthedocs.io/>. See {cite}`PyTorchGeometric` for more details.

In [1]:
import torch_geometric

In [2]:
# 48 track-level features
features = ['track_pt',
            'track_ptrel',
            'trackBTag_Eta',
            'trackBTag_DeltaR',
            'trackBTag_EtaRel',
            'trackBTag_JetDistVal',
            'trackBTag_Momentum',
            'trackBTag_PPar',
            'trackBTag_PParRatio',
            'trackBTag_PtRatio',
            'trackBTag_PtRel',
            'trackBTag_Sip2dSig',
            'trackBTag_Sip2dVal',
            'trackBTag_Sip3dSig',
            'trackBTag_Sip3dVal',
            'track_VTX_ass',
            'track_charge',
            'track_deltaR',
            'track_detadeta',
            'track_dlambdadz',
            'track_dlambdadz',
            'track_dphidphi',
            'track_dphidxy',
            'track_dptdpt',
            'track_drminsv',
            'track_drsubjet1',
            'track_drsubjet2',
            'track_dxy',
            'track_dxydxy',
            'track_dxydz',
            'track_dxysig',
            'track_dz',
            'track_dzdz',        
            'track_dzsig',
            'track_erel',
            'track_etarel',
            'track_fromPV',
            'track_isChargedHad',
            'track_isEl',
            'track_isMu',
            'track_lostInnerHits',
            'track_mass',
            'track_normchi2',            
            'track_phirel',
            'track_pt',
            'track_ptrel',
            'track_puppiw',
            'track_quality']

# spectators to define mass/pT window
spectators = ['fj_sdmass',
              'fj_pt']

# 2 labels: QCD or Hbb (we'll reduce the following labels)
labels =  ['label_QCD_b',
           'label_QCD_bb',
           'label_QCD_c', 
           'label_QCD_cc', 
           'label_QCD_others',
           'sample_isQCD',
           'label_H_bb']

nfeatures = len(features)
nspectators = len(spectators)

In [3]:
from GraphDataset import GraphDataset

In [4]:
graph_dataset = GraphDataset('data', features, labels, spectators, n_events=10000, n_events_merge=1000)

Processing...


  0%|          | 15/10000 [00:00<01:09, 144.56it/s]

21
35
24
32
28
24
25
32
49
31
40
42
16
34
25
42
25
21
32
39
12
31
31
29
15
18
37
16
17
15
16
32
32
26
19


  1%|          | 54/10000 [00:00<01:00, 164.06it/s]

10
19
10
27
30
27
26
37
11
29
11
14
43
39
27
42
41
31
27
12
21
31
40
16
15
39
25
24
33
25
29
31
36
12
40
23
42


...

 28%|██▊       | 2806/10000 [00:17<00:50, 142.10it/s]

11
23
58
28
18
33
22
32
63
19
35
36
19
23
18
40
26
29
26
37
14
14
31
16
27
17
19
47
14
22
18
21


 28%|██▊       | 2828/10000 [00:17<00:45, 158.46it/s]


41
24
22
10
24
6
35
19
0


ValueError: need at least one array to stack